In [1]:
from ase.io import read
import time
import sys
sys.path.append('../code')
from Generate_Descriptors import get_dscribe_descriptors

In [33]:
C7O2H10_structs = read("../data/C7O2H10.extxyz.gz", ":2000")

In [3]:
graphenes = read("../data/ManyGraphenes.extxyz.gz")

In [12]:
def TickTock(func, *args, **kwargs):
    tick = time.time()
    func_output = func(*args, **kwargs)
    tock = time.time()
    return func_output, tock - tick

In [34]:
species = ["C", "O", "H", "N", "Ca", "Li"]
rcut = 3
nmax = 12
lmax = 9
SoapList, SoapTime = TickTock(get_dscribe_descriptors, C7O2H10_structs, species = species, rcut = rcut, nmax = nmax, lmax = lmax, is_global=False, return_derivatives = False)
print(SoapTime)

23.963632822036743


In [41]:
species = ["C", "O", "H", "N", "Ca", "Li"]
rcut = 3
nmax = 12
lmax = 9
SoapList, SoapTime = TickTock(get_dscribe_descriptors, graphenes, species = species, rcut = rcut, nmax = nmax, lmax = lmax, is_global=False, return_derivatives = False)
print(SoapTime)

0.09846997261047363


In [44]:
from ase.build import bulk
diamond = bulk("C", 'diamond')
diamond
Structure3DPlot(diamond).Plot()

Atoms(symbols='C2', pbc=True, cell=[[0.0, 1.785, 1.785], [1.785, 0.0, 1.785], [1.785, 1.785, 0.0]])

In [43]:
class variable